<a href="https://colab.research.google.com/github/MitalPattani/M5-Forecasting-Accuracy/blob/master/top_down_approach_with_arima_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip install pmdarima

     |████████████████████████████████| 1.5MB 2.8MB/s 
     |████████████████████████████████| 2.1MB 16.5MB/s 
     |████████████████████████████████| 8.7MB 31.6MB/s 
  Found existing installation: Cython 0.29.21
    Uninstalling Cython-0.29.21:
      Successfully uninstalled Cython-0.29.21
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [3]:
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pandas.plotting import register_matplotlib_converters
from pmdarima.arima import auto_arima,OCSBTest,KPSSTest,ADFTest
from tqdm import tqdm
from sklearn import metrics
from statsmodels.tsa.ar_model import AR
import statsmodels.api as sm
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
# from stationarizer import simple_auto_stationarize
%matplotlib inline

In [4]:
path = ('./gdrive/My Drive/sales_prediction_accuracy/')
os.listdir(path)
calendar = pd.read_csv(path+"calendar.csv")
sales_data = pd.read_csv(path+"sales_train_validation.csv")
eval_data = pd.read_csv(path+"sales_train_evaluation.csv")
sell_price = pd.read_csv(path+"sell_prices.csv")
submissions = pd.read_csv(path+"sample_submission.csv")

In [5]:
# remove trends and/or seasonality from the data using OSCB test

def stationarize(one, counter=0):    
    try:
        trend_result = ADFTest().should_diff(one)[1]  
        if trend_result and counter<2:
            one = one.diff()
            one =one.dropna()
            counter += 1
            one = stationarize(one, counter)
    except:
        None
    return one

def season_stationary(one):
    try:
        season_result = OCSBTest(m=30).estimate_seasonal_differencing_term(one)
        if season_result:
            one = one.diff(30)
            one =one.dropna()
    except:
        None
    return one
  


In [6]:
sales_data['cat_id']  = (sales_data['store_id'] + '_' + sales_data['cat_id'])
sales_data['dept_id'] = (sales_data['store_id'] + '_' + sales_data['dept_id'])
sales_data['id'] = (sales_data['store_id'] + '_' + sales_data['id'])
days = ['d_'+str(i) for i in range(1,1549)]
sales_data = sales_data.drop(columns=days)

In [8]:
hierarchy = {0: "state_id", 1: "store_id", 2: "cat_id", 3: "dept_id", 4: "id"}

In [132]:
final_work = pd.DataFrame()

In [133]:
## make heirarchical dataframe 
for i in hierarchy:
  col = hierarchy[i]
  df_1 = sales_data.groupby(by = [col],as_index= False).sum()
  data_final = df_1.T
  data_final.columns = data_final.loc[col]
  data_final=data_final.drop(col)
  if final_work.empty:
    final_work = data_final.copy()
  else:
    final_work = pd.merge(final_work,data_final,left_index=True, right_index=True)
final_work['total'] = final_work['CA']+final_work['TX']+final_work['WI']

In [87]:
pred_days= ['f_'+ str(i) for i in range(1,29)]
result_sales = pd.DataFrame(columns=pred_days, index = final_work.index)

In [72]:
# sales_data_eval = remain.drop(columns = ['item_id','dept_id','cat_id','store_id','state_id']).copy()
# sales_data_eval = sales_data_eval.drop(columns = days)


In [ ]:
# make predictions for the entire heirarchy on each level

for i in tqdm(range(114), position=0, leave=True):
    one = final_work.iloc[i]
    id = final_work.index[i]
    # one = one.drop(['id'])
    one = stationarize(one)
    one = season_stationary(one)
    model_fit = auto_arima(one, m=1, suppress_warnings = True)
    pred = model_fit.predict(28)
    result_sales.loc[id] = pred

result_sales.dropna()
result_sales.to_csv('hier_preds.csv')


In [41]:
data_predictions = pd.read_csv('unchecked_final.csv')
heir = pd.read_csv('hier_preds.csv')
data_predictions['id'] = data_predictions['id'].str[-15:-11]+'_'+ data_predictions['id']
heir = heir.dropna() 
heir['id'] = heir['Unnamed: 0']
heir = heir.drop(columns='Unnamed: 0')
data_predictions = data_predictions.drop(columns='Unnamed: 0')
total_pred = pd.concat([data_predictions,heir], ignore_index=True)
num = total_pred._get_numeric_data()
num[num < 0] = 0
total_pred = total_pred.T
total_pred.columns = total_pred.loc['id']
total_pred = total_pred.drop('id')

In [42]:
total_pred.head()

id,CA_1_HOBBIES_1_001_CA_1_validation,CA_1_HOBBIES_1_002_CA_1_validation,CA_1_HOBBIES_1_003_CA_1_validation,CA_1_HOBBIES_1_004_CA_1_validation,CA_1_HOBBIES_1_005_CA_1_validation,CA_1_HOBBIES_1_006_CA_1_validation,CA_1_HOBBIES_1_007_CA_1_validation,CA_1_HOBBIES_1_008_CA_1_validation,CA_1_HOBBIES_1_009_CA_1_validation,CA_1_HOBBIES_1_010_CA_1_validation,CA_1_HOBBIES_1_011_CA_1_validation,CA_1_HOBBIES_1_012_CA_1_validation,CA_1_HOBBIES_1_013_CA_1_validation,CA_1_HOBBIES_1_014_CA_1_validation,CA_1_HOBBIES_1_015_CA_1_validation,CA_1_HOBBIES_1_016_CA_1_validation,CA_1_HOBBIES_1_017_CA_1_validation,CA_1_HOBBIES_1_018_CA_1_validation,CA_1_HOBBIES_1_019_CA_1_validation,CA_1_HOBBIES_1_020_CA_1_validation,CA_1_HOBBIES_1_021_CA_1_validation,CA_1_HOBBIES_1_022_CA_1_validation,CA_1_HOBBIES_1_023_CA_1_validation,CA_1_HOBBIES_1_024_CA_1_validation,CA_1_HOBBIES_1_025_CA_1_validation,CA_1_HOBBIES_1_026_CA_1_validation,CA_1_HOBBIES_1_027_CA_1_validation,CA_1_HOBBIES_1_028_CA_1_validation,CA_1_HOBBIES_1_029_CA_1_validation,CA_1_HOBBIES_1_030_CA_1_validation,CA_1_HOBBIES_1_031_CA_1_validation,CA_1_HOBBIES_1_032_CA_1_validation,CA_1_HOBBIES_1_033_CA_1_validation,CA_1_HOBBIES_1_034_CA_1_validation,CA_1_HOBBIES_1_035_CA_1_validation,CA_1_HOBBIES_1_036_CA_1_validation,CA_1_HOBBIES_1_037_CA_1_validation,CA_1_HOBBIES_1_038_CA_1_validation,CA_1_HOBBIES_1_039_CA_1_validation,CA_1_HOBBIES_1_040_CA_1_validation,...,TX_1_HOBBIES_1,TX_1_HOBBIES_2,TX_1_HOUSEHOLD_1,TX_1_HOUSEHOLD_2,TX_2_FOODS_1,TX_2_FOODS_2,TX_2_FOODS_3,TX_2_HOBBIES_1,TX_2_HOBBIES_2,TX_2_HOUSEHOLD_1,TX_2_HOUSEHOLD_2,TX_3_FOODS_1,TX_3_FOODS_2,TX_3_FOODS_3,TX_3_HOBBIES_1,TX_3_HOBBIES_2,TX_3_HOUSEHOLD_1,TX_3_HOUSEHOLD_2,WI_1_FOODS_1,WI_1_FOODS_2,WI_1_FOODS_3,WI_1_HOBBIES_1,WI_1_HOBBIES_2,WI_1_HOUSEHOLD_1,WI_1_HOUSEHOLD_2,WI_2_FOODS_1,WI_2_FOODS_2,WI_2_FOODS_3,WI_2_HOBBIES_1,WI_2_HOBBIES_2,WI_2_HOUSEHOLD_1,WI_2_HOUSEHOLD_2,WI_3_FOODS_1,WI_3_FOODS_2,WI_3_FOODS_3,WI_3_HOBBIES_1,WI_3_HOBBIES_2,WI_3_HOUSEHOLD_1,WI_3_HOUSEHOLD_2,total
f_1,1.04659,0.259537,0.776517,1.97324,1.44719,0.699735,0.284932,8.89189,0.733386,0.608219,0.0504268,0.227354,0.638704,1.94011,2.66993,6.33973,1.79498,0.00801946,8.10689,0.325407,0.69158,0.265753,1.08366,0.213699,0.541976,0.109084,0.433675,0.738069,1.96438,7.63615,0.371104,5.53532,0.29863,0.4981,0.156231,1.01927,0.734247,0.413699,0.145205,1.15189,...,248.852,4.46657,767.316,0,211.67,438.086,0,330.864,45.2279,814.257,0,258.02,450.099,0,337.667,10.9671,771.587,175.245,342,605.396,1724.71,345.146,0,583.281,185.353,344.98,994.574,2291.24,244.461,0,1003,149.061,275.555,560.934,2072.72,221.756,27.6332,629.541,0,41999.7
f_2,1.0486,0.273654,0.736554,1.11043,1.38708,0.698828,0.284932,9.35862,0.734275,0.608219,0.115943,0.22343,0.543647,1.91287,2.66351,6.33973,1.02906,0.00552506,8.05297,0.247928,0.910928,0.265753,1.23877,0.213699,0.690474,0.224472,0.434043,0.740159,1.96438,7.64638,0.371679,5.1875,0.29863,0.498095,0.165116,1.00878,0.734247,0.413699,0.145205,1.15417,...,248.852,0,661.498,0,203.177,380.279,0,312.844,45.2279,702.61,0,272.897,456.217,0,297.426,0,698.652,171.728,342,494.886,1342.61,271.043,0,483.69,168.811,350.207,984.747,2409.06,252.166,0,971.64,130.252,260.55,563.208,2018.93,223.421,29.8678,507.263,0,35798.1
f_3,1.05061,0.285596,0.754999,0.93181,1.33847,0.697921,0.284932,10.3775,0.735163,0.608219,0.151779,0.220348,0.317708,1.76974,2.65709,6.33973,1.02906,0.00622539,8.00154,0.254602,0.743776,0.265753,1.32489,0.213699,0.621719,0.327557,0.434411,0.742249,1.96438,7.65661,0.372255,4.98499,0.29863,0.498089,0.139286,0.502806,0.734247,0.413699,0.145205,1.15646,...,248.852,2.4672,637.368,15.8812,212.285,344.315,0,308.111,45.2279,651.994,0,277.412,460.492,0,281.65,0,673.55,171.728,342,472.694,1229.12,305.395,0,445.803,168.811,348.617,976.839,2298.02,252.302,0.870041,980.351,126.371,255.617,564.523,2024.11,214.717,29.7966,477.71,0,34170.8
f_4,1.05261,0.295699,0.698959,1.52821,1.29918,0.697014,0.284932,10.3546,0.736051,0.608219,0.0488665,0.217926,0.126638,1.67965,2.65067,6.33973,1.029

In [37]:
total_pred.to_csv('total_pred.csv')

In [38]:
hr = sales_data.iloc[:,0:6]
heir_pred_valid = pd.merge(hr,data_predictions,on = 'id')

In [45]:
pred_final = pd.DataFrame()

In [47]:
## rework the entire hierarchy after prediction 
for i in hierarchy:
  col = hierarchy[i]
  df_1 = heir_pred_valid.groupby(by = [col],as_index= False).sum()
  data_final = df_1.T
  data_final.columns = data_final.loc[col]
  data_final=data_final.drop(col)
  if pred_final.empty:
    pred_final = data_final.copy()
  else:
    pred_final = pd.merge(pred_final,data_final,left_index=True, right_index=True)
pred_final['total'] = pred_final['CA']+pred_final['TX']+pred_final['WI']

In [48]:
pred_final.head()

,CA,TX,WI,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3,CA_1_FOODS,CA_1_HOBBIES,CA_1_HOUSEHOLD,CA_2_FOODS,CA_2_HOBBIES,CA_2_HOUSEHOLD,CA_3_FOODS,CA_3_HOBBIES,CA_3_HOUSEHOLD,CA_4_FOODS,CA_4_HOBBIES,CA_4_HOUSEHOLD,TX_1_FOODS,TX_1_HOBBIES,TX_1_HOUSEHOLD,TX_2_FOODS,TX_2_HOBBIES,TX_2_HOUSEHOLD,TX_3_FOODS,TX_3_HOBBIES,TX_3_HOUSEHOLD,WI_1_FOODS,WI_1_HOBBIES,WI_1_HOUSEHOLD,WI_2_FOODS,WI_2_HOBBIES,WI_2_HOUSEHOLD,...,WI_3_HOUSEHOLD_2_478_WI_3_validation,WI_3_HOUSEHOLD_2_479_WI_3_validation,WI_3_HOUSEHOLD_2_480_WI_3_validation,WI_3_HOUSEHOLD_2_481_WI_3_validation,WI_3_HOUSEHOLD_2_482_WI_3_validation,WI_3_HOUSEHOLD_2_483_WI_3_validation,WI_3_HOUSEHOLD_2_484_WI_3_validation,WI_3_HOUSEHOLD_2_485_WI_3_validation,WI_3_HOUSEHOLD_2_486_WI_3_validation,WI_3_HOUSEHOLD_2_487_WI_3_validation,WI_3_HOUSEHOLD_2_488_WI_3_validation,WI_3_HOUSEHOLD_2_489_WI_3_validation,WI_3_HOUSEHOLD_2_490_WI_3_validation,WI_3_HOUSEHOLD_2_491_WI_3_validation,WI_3_HOUSEHOLD_2_492_WI_3_validation,WI_3_HOUSEHOLD_2_493_WI_3_validation,WI_3_HOUSEHOLD_2_494_WI_3_validation,WI_3_HOUSEHOLD_2_495_WI_3_validation,WI_3_HOUSEHOLD_2_496_WI_3_validation,WI_3_HOUSEHOLD_2_497_WI_3_validation,WI_3_HOUSEHOLD_2_498_WI_3_validation,WI_3_HOUSEHOLD_2_499_WI_3_validation,WI_3_HOUSEHOLD_2_500_WI_3_validation,WI_3_HOUSEHOLD_2_501_WI_3_validation,WI_3_HOUSEHOLD_2_502_WI_3_validation,WI_3_HOUSEHOLD_2_503_WI_3_validation,WI_3_HOUSEHOLD_2_504_WI_3_validation,WI_3_HOUSEHOLD_2_505_WI_3_validation,WI_3_HOUSEHOLD_2_506_WI_3_validation,WI_3_HOUSEHOLD_2_507_WI_3_validation,WI_3_HOUSEHOLD_2_508_WI_3_validation,WI_3_HOUSEHOLD_2_509_WI_3_validation,WI_3_HOUSEHOLD_2_510_WI_3_validation,WI_3_HOUSEHOLD_2_511_WI_3_validation,WI_3_HOUSEHOLD_2_512_WI_3_validation,WI_3_HOUSEHOLD_2_513_WI_3_validation,WI_3_HOUSEHOLD_2_514_WI_3_validation,WI_3_HOUSEHOLD_2_515_WI_3_validation,WI_3_HOUSEHOLD_2_516_WI_3_validation,total
f_1,14422.3,9214.66,10858.9,3836.7,3329.36,4854.06,2402.13,2862.36,3275.65,3076.65,3184.61,4502.18,3172.15,2344.83,525.378,966.495,1932.62,416.942,979.801,2680.92,573.426,1599.72,1536.13,369.323,496.684,1682.25,271.971,908.145,2046.46,355.795,873.394,1879.43,380.467,816.747,2051.97,403.36,729.281,3206.26,260.689,1035.23,...,0.090411,0.0794521,0.0949369,0.267891,0.0645918,0.589041,0.120446,0.147945,0.184807,0.0876712,0.167123,0.0739726,0.871233,0.453937,0.29701,0.255998,0.134247,1.65104,0.356164,0.103922,0.342466,0.421918,0.126027,0.417977,0.060274,0.111496,0.0289154,0.170552,0.0737692,0.448659,0.0767123,0.49589,0.472437,0.356164,0.276712,0.379343,-0.00076622,0.178082,0.0874659,34495.9
f_2,14423.5,9231.74,10769.5,3769.49,3405.87,4873.17,2374.94,2876.17,3241.73,3113.84,3169.38,4418.14,3182.01,2316.68,530.934,921.874,2022.76,416.63,966.48,2724.28,567.466,1581.43,1506.49,366.792,501.656,1717.74,268.807,889.627,2011.78,366.559,863.386,1902.09,380.697,831.055,2056.47,401.337,711.569,3103.9,264.308,1049.93,...,0.090411,0.0794521,0.0949369,0.301127,0.0630193,0.589041,0.311003,0.147945,0.175411,0.0876712,0.167123,0.0739726,0.871233,0.377684,0.237559,0.255998,0.134247,-0.148855,0.356164,0.103651,0.342466,0.421918,0.126027,0.417977,0.060274,0.0479566,0.0266394,0.163661,0.0989815,0.448659,0.0767123,0.49589,0.5443,0.356164,0.276712,0.379343,0.125569,0.178082,0.0831576,34424.7
f_3,14500.2,9271.51,10890.4,3743.18,3475.94,4902.01,2379.12,2878.28,3257.48,3135.75,3237.38,4435.42,3217.58,2322.02,528.152,893.007,2093.37,416.219,966.349,2767.96,569.232,1564.82,1506.78,373.237,499.098,1725.73,276.081,876.471,2024.84,365.707,866.933,1922.84,384.779,828.125,2116.98,393.315,727.086,3121.22,264.48,1049.72,...,0.090411,0.0794521,0.0949369,0.301127,0.0576004,0.589041,0.148153,0.147945,0.175411,0.0876712,0.167123,0.0739726,0.871233,0.367035,0.31124,0.255998,0.134247,-0.173935,0.356164,0.105813,0.342466,0.421918,0.126027,0.417977,0.060274,0.0705611,0.0295422,0.157704,0.0554292,0.448659,0.0767123,0.49589,0.555231,0.356164,0.276712,0.379343,0.0115036,0.178082,0.0730761,34662.1
f_4,14748.3,9362.9,10968.2,3820.75,3583.05,4978.49,236

In [50]:
pred_final.to_csv('all_sums.csv')

In [51]:
all_predi = pd.read_csv('total_pred.csv')
all_sum = pd.read_csv('all_sums.csv')

In [47]:
# hier_pred = [a for a in final_work.columns if a.endswith('validation')]

In [48]:
# final_work = final_work.drop(columns=hier_pred)

In [49]:
# final_work = final_work.T

In [57]:
all_predi.index = all_predi['Unnamed: 0']
all_sum.index = all_sum['Unnamed: 0']

all_predi = all_predi.drop(columns = ['Unnamed: 0'])
all_sum = all_sum.drop(columns = ['Unnamed: 0'])

In [59]:
states = sales_data.state_id.unique()
stores = sales_data.store_id.unique()
depts = sales_data.dept_id.unique()
cats = sales_data.cat_id.unique()
items = sales_data.id.unique()

# Here we build the tree as a dictionary. Each node (key in dict) has a list of 
# children value in dict, which in turn may also be a key in the dict, and have 
# children as well 
total = {'total': list(states)}
state_h = {k: [v for v in stores if v.startswith(k)] for k in states}
store_h = {k: [v for v in cats if v.startswith(k)] for k in stores}
dept_h = {k: [v for v in depts if v.startswith(k)] for k in cats}
item_h = {k: [v for v in items if v.startswith(k)] for k in depts}

In [73]:
hier_dict = {**total, **state_h, **store_h, **dept_h, **item_h}
hier_total = list(all_sum.columns[0:114])
hier_total.insert(0,'total')

In [75]:
# getting disaggregation proportions as mentioned by Hyndman

## p = Y/S 
# Y = predictions on that level
# S = sum of predictions of all levels below that level

def get_proportions(Y, S):
  Y_arr = Y.to_numpy(dtype='float32')
  S_arr = S.to_numpy(dtype='float32')
  P_arr = np.true_divide(Y_arr,S_arr, out=np.zeros_like(Y_arr), where=S_arr!=0)
  P = pd.Series(P_arr)
  return P

In [81]:
extremely_final = pd.DataFrame()
extremely_final['total'] = all_predi['total'] 

In [ ]:
# Now we disaggregate the proportions to each level

for i in hier_total:
  print('i ',i)
  p = get_proportions(extremely_final[i],all_sum[i])
  for j in hier_dict[i]:
    print('j ',j)
    x = all_predi[j].to_numpy(dtype='float32')
    y = p.to_numpy(dtype='float32')
    m = np.multiply(x,y)
    extremely_final[j] = pd.Series(m)

In [88]:
extremely_final.index= pred_days

In [94]:
extremely_final.head()

,total,CA,TX,WI,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3,CA_1_HOBBIES,CA_1_HOUSEHOLD,CA_1_FOODS,CA_2_HOBBIES,CA_2_HOUSEHOLD,CA_2_FOODS,CA_3_HOBBIES,CA_3_HOUSEHOLD,CA_3_FOODS,CA_4_HOBBIES,CA_4_HOUSEHOLD,CA_4_FOODS,TX_1_HOBBIES,TX_1_HOUSEHOLD,TX_1_FOODS,TX_2_HOBBIES,TX_2_HOUSEHOLD,TX_2_FOODS,TX_3_HOBBIES,TX_3_HOUSEHOLD,TX_3_FOODS,WI_1_HOBBIES,WI_1_HOUSEHOLD,WI_1_FOODS,WI_2_HOBBIES,WI_2_HOUSEHOLD,...,WI_3_HOUSEHOLD_2_477_WI_3_validation,WI_3_HOUSEHOLD_2_478_WI_3_validation,WI_3_HOUSEHOLD_2_479_WI_3_validation,WI_3_HOUSEHOLD_2_480_WI_3_validation,WI_3_HOUSEHOLD_2_481_WI_3_validation,WI_3_HOUSEHOLD_2_482_WI_3_validation,WI_3_HOUSEHOLD_2_483_WI_3_validation,WI_3_HOUSEHOLD_2_484_WI_3_validation,WI_3_HOUSEHOLD_2_485_WI_3_validation,WI_3_HOUSEHOLD_2_486_WI_3_validation,WI_3_HOUSEHOLD_2_487_WI_3_validation,WI_3_HOUSEHOLD_2_488_WI_3_validation,WI_3_HOUSEHOLD_2_489_WI_3_validation,WI_3_HOUSEHOLD_2_490_WI_3_validation,WI_3_HOUSEHOLD_2_491_WI_3_validation,WI_3_HOUSEHOLD_2_492_WI_3_validation,WI_3_HOUSEHOLD_2_493_WI_3_validation,WI_3_HOUSEHOLD_2_494_WI_3_validation,WI_3_HOUSEHOLD_2_495_WI_3_validation,WI_3_HOUSEHOLD_2_496_WI_3_validation,WI_3_HOUSEHOLD_2_497_WI_3_validation,WI_3_HOUSEHOLD_2_498_WI_3_validation,WI_3_HOUSEHOLD_2_499_WI_3_validation,WI_3_HOUSEHOLD_2_500_WI_3_validation,WI_3_HOUSEHOLD_2_501_WI_3_validation,WI_3_HOUSEHOLD_2_502_WI_3_validation,WI_3_HOUSEHOLD_2_503_WI_3_validation,WI_3_HOUSEHOLD_2_504_WI_3_validation,WI_3_HOUSEHOLD_2_505_WI_3_validation,WI_3_HOUSEHOLD_2_506_WI_3_validation,WI_3_HOUSEHOLD_2_507_WI_3_validation,WI_3_HOUSEHOLD_2_508_WI_3_validation,WI_3_HOUSEHOLD_2_509_WI_3_validation,WI_3_HOUSEHOLD_2_510_WI_3_validation,WI_3_HOUSEHOLD_2_511_WI_3_validation,WI_3_HOUSEHOLD_2_512_WI_3_validation,WI_3_HOUSEHOLD_2_513_WI_3_validation,WI_3_HOUSEHOLD_2_514_WI_3_validation,WI_3_HOUSEHOLD_2_515_WI_3_validation,WI_3_HOUSEHOLD_2_516_WI_3_validation
f_1,41999.723748,23209.884766,13419.338867,15265.565430,7898.325684,0.000000,0.000000,0.000000,0.000000,5960.094238,5434.043945,5074.375977,7148.223633,6080.109863,1191.370728,2078.260254,6758.569336,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,672.125732,1893.197510,0.000000,739.761108,1650.093506,4308.176758,652.692749,1161.495361,3817.913330,431.721252,1875.350830,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
f_2,35798.098078,16444.363281,10606.299805,11155.178711,4534.138184,0.000000,0.000000,0.000000,0.000000,4118.905762,4015.523682,3259.933105,5437.143066,4479.874023,654.263733,985.880188,3171.305908,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,442.428833,1163.170898,0.000000,523.194397,1134.339355,2866.748779,332.532410,602.921997,2136.373291,343.217560,1422.978149,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
f_3,34170.770282,14393.207031,9820.426758,10082.069336,3680.550537,81.558792,0.000000,0.000000,0.000000,3526.286377,3614.248779,2968.280273,4815.568848,4003.985840,534.826050,786.571411,2424.498291,7.847678,20.734341,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,374.437988,919.997864,0.000000,467.621552,881.319519,2462.023926,302.420502,509.225250,1972.881348,304.230682,1242.447144,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [92]:
extremely_final.to_csv('extremely_final.csv')